In [29]:
# !pip install import_ipynb --user
import pandas as pd
from catboost import CatBoostClassifier
from sklearn import preprocessing
import pickle
import import_ipynb
import numpy as np

In [62]:
users = pd.read_pickle('../datasets/UsersDataset.pkl')
users.head(3)

,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
index,,,,,,,,,,,,,,,
2300560809,2300560809,NMoraniB,Nicole Morani Brown,593,Gerencia General de @PuertoBsAs Ministerio de ...,"Buenos Aires, Argentina",False,False,0,2014-01-01 00:00:00,2019-06-29 23:04:21.059135,2377,1630,1510,False
1536923558,1536923558,FabianPereyra87,Fabian Pereyra,2532,,False,False,False,0,2013-06-01 00:00:00,2019-06-29 23:04:21.289518,4307,53,284,False
235491971,235491971,gsrcaballero,Gus Caballero,21635,"Papa de Cami y Rochi, FANA de La Academia, PRO...",False,False,False,0,2011-01-01 00:00:00,2019-06-29 23:04:21.030207,5683,1691,1360,False


In [56]:
# Cargar Modelo
# model = CatBoostClassifier().load_model("../modelo/modelo")

# import os
# os.chdir("../modelo/")
# from generacionModelo import transform_x
# os.chdir("../dataAnalysis/")

# from '../modelo/Model UserBasicClassifier.ipynb' import transform_x

def transform_x(df):
    df.loc[:, 'crawled_at'] = pd.DatetimeIndex(pd.to_datetime(df.crawled_at, utc=None)).tz_convert(tz='UTC')
    df.loc[:, 'created_at'] = pd.DatetimeIndex(pd.to_datetime(df.created_at, utc=None)).tz_convert(tz='UTC')
    
    
    account_life = df.crawled_at - df.created_at
    account_life_meses = np.round(account_life.dt.days / 30)
    account_life_meses = account_life_meses.replace(0, 1)
    account_life_semanas = np.round(account_life.dt.days / 7)
    account_life_semanas = account_life_semanas.replace(0, 1)
    numValues = ['tweet_count', 'listed_count', 'favs_count', 'followers_count', 'following_count']
    for i in numValues:
        colName_mes = i+'_mes'
        df.loc[:,colName_mes] = df[i].astype(float) / account_life_meses

        colName_semana = i+'_semana'
        df.loc[:,colName_semana] = df[i].astype(float) / account_life_semanas

    df.loc[:,'verified'] = ~df.verified.isna()
    df.loc[:,'description'] = ~df.description.isna()
    df.loc[:,'location'] = ~df.location.isna()
    df.loc[:,'url'] = ~df.url.isna()

    columnas = np.concatenate(
        [[x, x+'_mes', x+'_semana'] for x in numValues]
    )
    columnas = np.concatenate([
        [
            'verified',
            'description',
            'location',
            'url',
        ],
        columnas
    ])
    df = df[columnas]
    df = df.fillna(0)
    return df


transform_x(users).head()

,verified,description,location,url,tweet_count,tweet_count_mes,tweet_count_semana,listed_count,listed_count_mes,listed_count_semana,favs_count,favs_count_mes,favs_count_semana,followers_count,followers_count_mes,followers_count_semana,following_count,following_count_mes,following_count_semana
index,,,,,,,,,,,,,,,,,,,
2300560809,True,True,True,True,593,8.850746,2.073427,0,0.0,0.0,2377,35.477612,8.311189,1630,24.328358,5.699301,1510,22.537313,5.279720
1536923558,True,True,True,True,2532,34.216216,7.987382,0,0.0,0.0,4307,58.202703,13.586751,53,0.716216,0.167192,284,3.837838,0.895899
235491971,True,True,True,True,21635,210.048544,48.837472,0,0.0,0.0,5683,55.174757,12.828442,1691,16.417476,3.817156,1360,13.203883,3.069977
948739166,True,True,True,True,786,9.703704,2.265130,0,0.0,0.0,346,4.271605,0.997118,132,1.629630,0.380403,580,7.160494,1.671470
497428913,True,True,True,True,2256,25.066667,5.844560,0,0.0,0.0,4252,47.244444,11.015544,146,1.622222,0.378238,130,1.444444,0.336788


In [9]:
with open('../modelo/modelPipeline.pkl', 'rb') as m_pred:
    model = pickle.load(m_pred)

model

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('cat', <catboost.core.CatBoostClassifier object at 0x000002753D83D6D8>)])

In [58]:
predicciones = model.predict_proba(transform_x(users))

In [67]:
# Joinear con nombres de usuarios, limpiar indices y joinear por orden de aparicion
usuarios_prob = pd.DataFrame(users.screen_name.to_list()).join(pd.DataFrame(predicciones)[1])
usuarios_prob.columns = ['Usuario','Prob_Bot']
# Quedarnos con los que dan probabilidad mayor al 99% de ser bots segun el modelo basico
usuarios_prob = usuarios_prob[usuarios_prob['Prob_Bot']>0.70]
usuarios_prob.sort_values(by='Prob_Bot',ascending=False)

,Usuario,Prob_Bot
6223,Moka_Fin,0.997794
5158,infpolitico,0.996354
4661,Mario_PSCL,0.996232
3958,PedroFernandez1,0.995051
3912,ElNacionalNoti,0.994137
2729,LuzzSaucedo,0.993758
6544,hay2019arg,0.993748
969,LucasPodversic,0.993218
5961,RicardoIorio_,0.992459
6516,pagina12noof,0.991629
